In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on a hardware accelerator like the
GPU or MPS, if available. Let\'s check to see if
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) or
[torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html) are
available, otherwise we use the CPU.


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


* MPS是苹果的框架


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [12]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


In [14]:
for name, param in model.state_dict().items():
    print(f"Layer: {name}")
    print(f"Parameters: {param}")
    print("-" * 50)

Layer: linear_relu_stack.0.weight
Parameters: tensor([[-0.0049,  0.0128,  0.0318,  ...,  0.0173, -0.0204, -0.0351],
        [ 0.0202,  0.0038,  0.0138,  ..., -0.0012,  0.0189,  0.0086],
        [-0.0170,  0.0153,  0.0129,  ...,  0.0165,  0.0191, -0.0120],
        ...,
        [ 0.0020,  0.0241,  0.0249,  ...,  0.0232,  0.0273,  0.0239],
        [-0.0303, -0.0126,  0.0168,  ..., -0.0135,  0.0193,  0.0053],
        [ 0.0052, -0.0091, -0.0306,  ...,  0.0221, -0.0117, -0.0184]],
       device='cuda:0')
--------------------------------------------------
Layer: linear_relu_stack.0.bias
Parameters: tensor([ 2.6383e-02, -3.5446e-02,  1.3896e-02, -2.0674e-02,  1.9225e-02,
        -3.2090e-02,  1.6844e-02, -1.7002e-03, -1.5262e-02,  8.9034e-03,
         8.4785e-03, -2.5233e-02,  2.3913e-02, -2.5272e-02,  1.4886e-02,
         4.0631e-04,  3.5307e-02, -2.6608e-02,  5.7709e-03,  2.6886e-02,
        -3.2420e-03,  1.0908e-02,  3.0316e-02, -1.5200e-03, -2.5438e-02,
         1.5165e-02, -8.4056e-03, -5

------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [15]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [16]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Flatten()是从第二维开始展开的！

nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [18]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [19]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.5607, -0.1083,  0.0623,  0.3736,  0.2574,  0.1707, -0.1046, -0.5103,
          0.1268, -0.0930,  0.2521,  0.1772, -0.0455,  0.1547,  0.1576,  0.0492,
          0.0445,  0.0026,  0.5106, -0.0332],
        [-0.5189,  0.0398,  0.1567,  0.3442,  0.2293,  0.4436, -0.2753, -0.9907,
          0.4714, -0.1631, -0.2132,  0.1636,  0.2717,  0.1548,  0.3589, -0.1715,
         -0.2148,  0.3832,  0.4935, -0.3987],
        [-0.5771, -0.0132,  0.0528,  0.6429,  0.0387,  0.5962, -0.3136, -0.4916,
          0.7058, -0.2945,  0.0137,  0.1006, -0.0687, -0.1460,  0.7408,  0.0595,
         -0.3037, -0.0662,  0.3911, -0.3258]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0623, 0.3736, 0.2574, 0.1707, 0.0000, 0.0000, 0.1268,
         0.0000, 0.2521, 0.1772, 0.0000, 0.1547, 0.1576, 0.0492, 0.0445, 0.0026,
         0.5106, 0.0000],
        [0.0000, 0.0398, 0.1567, 0.3442, 0.2293, 0.4436, 0.0000, 0.0000, 0.4714,
         0.0000, 0.0000, 0.1636, 0.2717, 0.1548, 0.35

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [20]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

* nn.Linear(20, 10) 前面是输入的维度，后面是输出的维度

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
